In [1]:
import requests
import xml.etree.ElementTree as ET
import logging
import urllib.parse 
import pandas as pd
from io import StringIO
from io import BytesIO

import warnings
import urllib3
import json
warnings.simplefilter("ignore")
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

config_file_path = 'api.json'
config_data = {}

with open(config_file_path, 'r') as f:
    config_data = json.load(f)
    username = config_data.get('tableau_username')
    password = config_data.get('tableau_password')
    server_url = config_data.get('tableau_server_url')
    api_version = config_data.get('api_version')
    content_url = config_data.get('site_id')

In [2]:
print(config_data)

{'tableau_username': 'aranja14@ext.uber.com', 'tableau_password': 'Xtreme@7864Xtreme', 'tableau_server_url': 'https://tableau-aws.uberinternal.com', 'api_version': '3.19', 'site_id': 'CODS'}


In [3]:
def sign_in(server_url, api_version, username, password, content_url=""):
    url = f"{server_url}/api/{api_version}/auth/signin"
    headers = {"Content-Type": "application/xml"}
    payload = f"""
        <tsRequest>
            <credentials name="{username}" password="{password}">
                <site contentUrl="{content_url}"/>
            </credentials>
        </tsRequest>
    """
    response = requests.post(url, headers=headers, data=payload, verify=False)
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        namespace = {'ts': 'http://tableau.com/api'}
        auth_token = root.find(".//ts:credentials", namespace).attrib['token']
        site_id = root.find(".//ts:site", namespace).attrib['id']
        return auth_token, site_id
    else:
        raise Exception(f"Failed to sign in (Status Code: {response.status_code}): {response.text}")

In [4]:
def get_workbook_id(server_url, api_version, site_id, auth_token, workbook_name, verify_ssl=False):
    if not auth_token:
        raise Exception("Authentication token is missing. Cannot get workbook ID.")
    encoded_workbook_name = urllib.parse.quote_plus(workbook_name)
    url = f"{server_url}/api/{api_version}/sites/{site_id}/workbooks?filter=name:eq:{encoded_workbook_name}"
    headers = {"X-Tableau-Auth": auth_token}
    response = requests.get(url, headers=headers, verify=verify_ssl)
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        namespace = {'ts': 'http://tableau.com/api'}
        workbook_element = root.find(".//ts:workbook", namespace)
        if workbook_element is not None:
            workbook_id = workbook_element.attrib['id']
            return workbook_id
        else:
            raise Exception(f"Workbook '{workbook_name}' not found on site '{site_id}'. Response: {response.text}")
    else:
        raise Exception(f"Failed to fetch workbooks (Status Code: {response.status_code}): {response.text}")


In [5]:
def get_workbook_views(server_url, api_version, site_id, workbook_id, auth_token, verify_ssl=False):
    if not auth_token:
        raise Exception("Authentication token is missing. Cannot get workbook sheets.")

    url = f"{server_url}/api/{api_version}/sites/{site_id}/workbooks/{workbook_id}/views"
    headers = {"X-Tableau-Auth": auth_token}

    response = requests.get(url, headers=headers, verify=verify_ssl)

    if response.status_code == 200:
        root = ET.fromstring(response.text)
        namespace = {'ts': 'http://tableau.com/api'}
        
        views_data = []
        for view_elem in root.findall(".//ts:view", namespace):
            view_info = {
                'id': view_elem.attrib.get('id'),
                'name': view_elem.attrib.get('name'),
                'contentUrl': view_elem.attrib.get('contentUrl'),
                'workbookId': view_elem.attrib.get('workbookId'),
                'ownerId': view_elem.attrib.get('ownerId'),
                'totalViews': int(view_elem.attrib.get('totalViews', 0)), # Convert to int
                'createdAt': view_elem.attrib.get('createdAt'),
                'updatedAt': view_elem.attrib.get('updatedAt')
            }
            views_data.append(view_info)
        
        return views_data
    else:
        raise Exception(f"Failed to get workbook sheets (Status Code: {response.status_code}): {response.text}")


In [6]:
def get_embedded_sheets_in_view(server_url, auth_token, dashboard_id, verify_ssl=False):
    metadata_url = f"{server_url}/api/metadata/graphql"
    
    headers = {
        "X-Tableau-Auth": auth_token,
        "Content-Type": "application/json",
        "Accept": "application/json" 
    }
    graphql_query = f"""
    query GetDashboardSheets {{
      dashboards(filter: {{luid: "{dashboard_id}"}}) {{
        name
        luid
        sheets {{
          luid
          name
          path # This can be useful for debugging/verification
        }}
      }}
    }}
    """
    payload = json.dumps({"query": graphql_query})

    logger.info(f"Metadata API URL: {metadata_url}")
    logger.info(f"GraphQL Query (partial): {graphql_query[:100]}...")

    response = requests.post(metadata_url, headers=headers, data=payload, verify=verify_ssl)
    logger.info(f"Metadata API Response Status: {response.status_code}")

    if response.status_code == 200:
        response_json = response.json()

        if 'errors' in response_json:
            raise Exception(f"Metadata API GraphQL Errors: {response_json['errors']}")

        dashboards_data = response_json.get('data', {}).get('dashboards', [])
        
        embedded_sheets_info = []
        if dashboards_data:
            dashboard = dashboards_data[0]
            sheets = dashboard.get('sheets', [])
            for sheet in sheets:
                embedded_sheets_info.append({
                    'id': sheet.get('luid'), 
                    'name': sheet.get('name'),
                    'path': sheet.get('path')
                })
            logger.info(f"Found {len(embedded_sheets_info)} embedded sheets for dashboard '{dashboard.get('name')}'")
        else:
            logger.warning(f"Dashboard with ID '{dashboard_id}' not found in Metadata API response.")
            
        return embedded_sheets_info
    else:
        raise Exception(f"Failed to query Metadata API (Status Code: {response.status_code}): {response.text}")


In [7]:
def prepare_view_filter_query(filters):
    if not filters:
        return ""
    filter_params = []
    for field_name, value in filters.items():
        encoded_field_name = urllib.parse.quote_plus(field_name)
        if isinstance(value, list):
            encoded_value = ",".join([urllib.parse.quote_plus(str(v)) for v in value])
        else:
            encoded_value = urllib.parse.quote_plus(str(value))
        filter_params.append(f"vf_{encoded_field_name}={encoded_value}")
    return "&".join(filter_params)

In [8]:
def get_view_crosstab_data(server_url, api_version, site_id, view_id, auth_token, filters):
    logger.info(f"get_view_data(view_id={view_id})")
    filter_query = prepare_view_filter_query(filters)
    url = f"{server_url}/api/{api_version}/sites/{site_id}/views/{view_id}/data"
    if filter_query:
        url += '?' + filter_query
    headers = {"X-Tableau-Auth": auth_token}
    logger.info(f"Request URL: {url}")
    response = requests.get(url, headers=headers, verify=False)
    logger.info(f"Response: {response}")
    if response.status_code == 200:
        logger.info(f"Received response text (first 500 chars): {response.text[:500]}...")
        csv_data = StringIO(response.text)
        logger.info(f"StringIO object created from response text.")
        df = pd.read_csv(csv_data)
        logger.info(f"DataFrame successfully created with {len(df)} rows.")
    else:
        df = None
        raise Exception(f"Failed to get view data (Status Code: {response.status_code}): {response.text}")
    return df

In [9]:
auth_token, site_id = sign_in(server_url,api_version,username,password,content_url)
print(f"Auth Token: {auth_token}")
print(f"Site ID: {site_id}")

Exception: Failed to sign in (Status Code: 401): <?xml version='1.0' encoding='UTF-8'?><tsResponse xmlns="http://tableau.com/api" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://tableau.com/api https://help.tableau.com/samples/en-us/rest_api/ts-api_3_19.xsd"><error code="401001"><summary>Signin Error</summary><detail>Error signing in to Tableau Server</detail></error></tsResponse>

In [9]:
workbook_name = "Ads Delivery Finance Dashboard"
workbook_id = get_workbook_id(server_url,api_version,site_id,auth_token,workbook_name,verify_ssl=False)
print(f"Workbook ID: {workbook_id}")

NameError: name 'site_id' is not defined

In [10]:
views_in_workbook = get_workbook_views(server_url,api_version,site_id,workbook_id,auth_token,verify_ssl=False)
print(views_in_workbook)

[{'id': '72d9f138-e4c5-4027-a6e8-a955f36f36a7', 'name': 'ANR', 'contentUrl': 'AdsDashboard/sheets/ANR', 'workbookId': None, 'ownerId': None, 'totalViews': 0, 'createdAt': '2025-07-14T16:05:28Z', 'updatedAt': '2025-07-15T17:29:41Z'}, {'id': '68a2e492-2980-4c24-86bf-82c16ae76ac3', 'name': 'Ads Delivery Finance Dashboard', 'contentUrl': 'AdsDashboard/sheets/Dashboard1', 'workbookId': None, 'ownerId': None, 'totalViews': 0, 'createdAt': '2025-02-10T18:08:16Z', 'updatedAt': '2025-07-15T17:29:41Z'}, {'id': '71441011-ab4d-4348-9b67-10d3fc707fe3', 'name': 'Ads Delivery Finance All Metrics Panel', 'contentUrl': 'AdsDashboard/sheets/Dashboard2', 'workbookId': None, 'ownerId': None, 'totalViews': 0, 'createdAt': '2025-05-13T16:50:34Z', 'updatedAt': '2025-07-15T17:29:41Z'}]


In [11]:
if views_in_workbook:
    for i, sheet in enumerate(views_in_workbook):
        print(f" Sheet {i+1}:")
        print(f" Name: {sheet.get('name')}")
        print(f" ID: {sheet.get('id')}")
        print(f" Content URL: {sheet.get('contentUrl')}")
        dashboard_id = sheet.get('id')
        embedded_sheets = get_embedded_sheets_in_view(server_url,auth_token,dashboard_id, verify_ssl=False)
        print(embedded_sheets)
else:
    print("No sheets/views found in this workbook.")

 Sheet 1:
 Name: ANR
 ID: 72d9f138-e4c5-4027-a6e8-a955f36f36a7
 Content URL: AdsDashboard/sheets/ANR
[]
 Sheet 2:
 Name: Ads Delivery Finance Dashboard
 ID: 68a2e492-2980-4c24-86bf-82c16ae76ac3
 Content URL: AdsDashboard/sheets/Dashboard1
[{'id': '', 'name': 'clicks', 'path': ''}, {'id': '', 'name': 'CTR', 'path': ''}, {'id': '', 'name': 'Impressions', 'path': ''}, {'id': '', 'name': 'Ads Offer Active Merchants', 'path': ''}, {'id': '', 'name': 'ROAS', 'path': ''}, {'id': '', 'name': 'Billed Ad Spend', 'path': ''}, {'id': '', 'name': 'Ads Active Merchants', 'path': ''}, {'id': '', 'name': 'Gross Booking', 'path': ''}, {'id': '', 'name': 'Resto Promo Spend', 'path': ''}, {'id': '', 'name': 'Budget Utilization', 'path': ''}, {'id': '', 'name': 'Storefront Tap', 'path': ''}, {'id': '', 'name': 'MPF Collected', 'path': ''}, {'id': '', 'name': 'Ad Credit', 'path': ''}, {'id': '', 'name': 'Ads Penetration Merchants', 'path': ''}, {'id': '', 'name': 'Ad Spend', 'path': ''}, {'id': '', 'name':

In [12]:
view_id = "68a2e492-2980-4c24-86bf-82c16ae76ac3"
crosstab_df = get_view_crosstab_data(server_url,api_version,site_id,view_id,auth_token,filters=None)
crosstab_df

,"Month, Day, Year of datestr",p. Breakdown by,ads_net_revenue_usd
0,"January 6, 2025",Global,"23,472,911"
1,"January 13, 2025",Global,"24,801,870"
2,"January 20, 2025",Global,"24,979,019"
3,"January 27, 2025",Global,"24,423,124"
4,"February 3, 2025",Global,"24,050,601"
5,"February 10, 2025",Global,"24,289,103"
6,"February 17, 2025",Global,"25,230,178"
7,"February 24, 2025",Global,"21,695,344"
8,"March 3, 2025",Global,"24,086,902"
9,"March 10, 2025",Global,"25,116,724"


In [ ]:
tableau_username"aranja14@ext.uber.com"
tableau_password"Xtreme@7864Xtreme"
tableau_server_url"https://tableau-aws.uberinternal.com"
api_version"3.19"
site_id"GlobalAdOps"